# Onebox helper

**Usage profile:**
- start search center
- preferred user language

**Results limits:**
```
OneBoxMap.default_terms_limit = 3
OneBoxMap.default_results_limit = 10
OneBoxMap.default_suggestions_limit = 5
```

In [ ]:
from search.widgets import OneBoxMap, Design, SearchResultButtons

app = OneBoxMap(language='en', latitude=40.71455, longitude=-74.00714, 
          design=Design.three, 
          resultlist_class=SearchResultButtons)
app.run()

## Various designs

`Design.two` and `SearchResultJson`

In [ ]:
from search.widgets import SearchResultJson

app = OneBoxMap(language='en', latitude=40.71455, longitude=-74.00714, 
          design=Design.one, 
          resultlist_class=SearchResultJson)
app.run()

## Adapt query parser

`OneBoxExt` and category near category

In [ ]:
from search.widgets import OneBoxExt

app = OneBoxExt(language='en', latitude=40.71455, longitude=-74.00714, 
          design=Design.three, 
          resultlist_class=SearchResultButtons)
app.run()